### KEN 3140: Lab 3 (RDFS reasoning)
#### Learning objectives:
1. Be able to identify which knowledge representation features are not included in [Resource Description Framework (RDF)](https://www.w3.org/TR/rdf11-concepts/) and are new to its extension: [RDFS](https://www.w3.org/TR/rdf-schema/)
2. Be able to identify and add suitable [domain](https://www.w3.org/TR/rdf-schema/#ch_domain), [range](https://www.w3.org/TR/rdf-schema/#ch_range) and [subclass](https://www.w3.org/TR/rdf-schema/#ch_subclassof) relations to an RDF graph to capture more detailed meaning for the entities in the graph
3. Be able to apply RDFS reasoning services to an RDF graph to compute implicit inferences about the graph which are not explicitly indicated

### 1. Background
#### A. RDF recap 
As we saw in Lectures 2 and 3 in the course, the [Resource Description Framework (RDF)](https://www.w3.org/TR/rdf11-concepts/) provides a way to capture information about a domain of interest as sets of statements called "triples", each of which is composed of three parts: [a subject, predicate and object](https://www.w3.org/TR/rdf11-concepts/#data-model).

We also saw that subjects, predicates and objects are denoted by [Internationalized Resource Identifiers (IRIs)](https://tools.ietf.org/html/rfc3987). One of the main reasons for using IRIs is to make it easier to assign **globally unique** names for the entities so we can eliminate the possibility of another resource somewhere on the Web using the same name, causing confusion as to which entity the identifer refers to. 

Another reason is of course related to the [Linked Data paradigm](https://www.w3.org/wiki/LinkedData)! Hopefully, [HTTP](https://www.w3.org/Protocols/#rfc723x) IRIs are used so that there can eventually be webpages published at these IRIs to describe information about the associated entity with links to other related entities on the Web.

#### B. RDF Schema recap
We recall that with RDF we can represent relationships between "things" (also called instances and objects). For example, in RDF I can state that "John is married to Mary". One possible triple representation of this could be 

``<http://somenamespace.com/john> <http://dbpedia.org/ontology/spouse> <http://somenamespace.com/mary>``.

But how does a machine know what "john" and "mary" are? Suppose we know that John is a chef and Mary is an Engineer. We can state that with these triples in RDF:

``<http://somenamespace.com/john> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Chef>``.

``<http://somenamespace.com/mary> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/Engineer>``.

While it may seem obvious to us that chefs and engineers are people, a machine does not find this obvious. Inferring that Mary and John are people would enable the machine to make other inferences which may be helpful. Therefore, we need a way to state this in RDF. We can of course add new triples to the graph (analogous to the ones above) to state that John and Mary are persons. However, this approach leaves out the expression of how the types "chef" and "engineer" relate to the type "person".

##### i. rdfs:subClassOf
The good news is that RDF Schema extends RDF to provide a relation called [rdfs:subClassOf](https://www.w3.org/TR/rdf-schema/#ch_subclassof) which allows us to state that some types are subclasses of other types. Of course, you could have created your own new custom relation to state subclass relations, however this approach will not define the **machine-interpretable meaning** of the relation. RDFS, on the other hand, [defines its types and relations mathematically](https://www.w3.org/TR/rdf11-mt/).

For our example about John and Mary, we could add the following RDFS triples to our graph to solve the problem: 

``<http://dbpedia.org/ontology/Chef> <http://www.w3.org/2000/01/rdf-schema#subClassOf> <http://dbpedia.org/ontology/Person>``.

``<http://dbpedia.org/ontology/Engineer> <http://www.w3.org/2000/01/rdf-schema#subClassOf> <http://dbpedia.org/ontology/Person>``.

Intuitively, these statements mean that "every instance of the Chef type is also an instance of the Person type" and "every instance of the Engineer type is also an instance of the Person type", respectively. It is not required for this course to understand the semantics of RDFS in full technical detail. However, it is important to grasp the intuition behind the semantics and to know that RDFS statements are not just "rules" as you would find in logic programming formalisms such as [Prolog](https://en.wikipedia.org/wiki/Prolog). **Side note:** the semantics for RDFS as well as for [OWL](https://www.w3.org/TR/owl2-overview/) are [model-theoretic](https://plato.stanford.edu/entries/model-theory/) (we will touch upon this in the course when we get to OWL).

##### ii. rdfs:subPropertyOf

[rdfs:subPropertyOf](https://www.w3.org/TR/rdf-schema/#ch_subpropertyof) is a similar property to rdfs:subClassOf, but for **predicates**. The intuition is to capture knowledge such as: "if a person x is married to a person y" then it should also be the case that "x knows y". This can be represented with the RDFS triple: 

``<http://dbpedia.org/ontology/spouse> <http://www.w3.org/2000/01/rdf-schema#subPropertyOf> <http://xmlns.com/foaf/spec/#term_knows>``

Of course, the other direction is not necessarily true (i.e. if "x knows y" it does not mean that "x is married to y") so we do **not** add another triple to state that [foaf:knows](http://xmlns.com/foaf/spec/#term_knows) [rdfs:subPropertyOf](https://www.w3.org/2000/01/rdf-schema#subPropertyOf) [dbo:spouse](http://dbpedia.org/ontology/spouse). 

Being able to state sub-properties is useful in RDFS because it enables the reasoner to infer useful implicit information. E.g. if I have a triple which says that 

##### iii. rdfs:domain & rdfs:range

[rdfs:domain](https://www.w3.org/TR/rdf-schema/#ch_domain) is a property that you can assign to a predicate in RDFS. Given a predicate, say p, the domain of p is a **type** which each **subject** must belong to in each triple for which p is the predicate. For example, consider a predicate "employs" which intuitively captures the relation of some entity employing another. That is, some entity that pays another to work for them. In this case, the domain of the "employs" predicate should arguably be some type of organisation. That is, we want to say that only organisations can employ things. An example of how this can be done with RDFS is (we make use of Schema.org vocabulary):

``<http://schema.org/employee> <http://www.w3.org/2000/01/rdf-schema#domain> <http://schema.org/Organization>``

[rdfs:range](https://www.w3.org/TR/rdf-schema/#ch_range) is to **objects** of triples what [rdfs:domain](https://www.w3.org/TR/rdf-schema/#ch_domain) is to **subjects** of triples. It is a property you can assign to RDF/RDFS predicates that constrain the types of **objects** in triples. Using the "employs" relation above as an example, the range of this relation should arguably be persons. I.e., by adding the following triple we constrain the "employs" relation to only be able to end in instances of type person:

``<http://schema.org/employee> <http://www.w3.org/2000/01/rdf-schema#range> <http://schema.org/Person>``

Both domain and range constrains together enforce in our graph that whenever x employs y occurs, x must be of type Organization and y must be of type Person. Can you think of examples which violate these constraints?

##### iv. rdfs:label, rdfs:comment
[rdfs:label](https://www.w3.org/TR/rdf-schema/#ch_label) and [rdfs:comment](https://www.w3.org/TR/rdf-schema/#ch_comment) are non-semantic predicates included in RDFS to annotate entities in the graph. rdfs:label allows one to assign a human-readable name (label) to a resource. rdfs:comment enables one to attach a piece of text to a resource which either describes what the resource is or describes some important relevant information about it. rdfs:label values are generally shorter pieces of text, and rdfs:comment values are usually longer textual descriptions. Look at the [spouse resource page](http://dbpedia.org/ontology/spouse) on [DBpedia](https://wiki.dbpedia.org/about) to see an example of the usage and distinction between these two predicates. 

### 2. Setup rdflib and owlrl
We will be using [RDFlib](https://github.com/RDFLib/rdflib) for parsing the Turtle(*.ttl) files and [OWLRL](https://github.com/RDFLib/OWL-RL/) for reasoning. Both packages are open source and can be installed using pip.

In [1]:
%pip install rdflib
%pip install owlrl

     |████████████████████████████████| 501kB 7.8MB/s eta 0:00:01
     |████████████████████████████████| 102kB 2.2MB/s eta 0:00:01
     |████████████████████████████████| 51kB 981kB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.
     |████████████████████████████████| 61kB 1.2MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


### 3. Start of the lab!
We are going to use a simple RDF graph using the additional features of RDFS to demonstrate the reasoning services of RDFS. In the text editor of your choice / in Jupyter, open and study the RDF graph located in the file ``Lab3_RDFS_graph.ttl``. The graph is represented in Turtle syntax. In Turtle syntax, comments are preceded by the ``#`` character so everything after this character on a particular line is ignored by RDF parsers. You will notice there are two sections titled "Schema" and "Data" respectively using Turtle comments. The triples under the section titled "Schema" are currently all commented out.

#### A. Build RDFS model and print triples
Run the code below to build an RDFS graph from the file ``Lab3_RDFS_graph.ttl``

In [8]:
import rdflib as rdf
import owlrl as owlrl
def print_graph(graph : rdf.Graph, resource, property, otherResource):
    for s, p, o in graph.triples((resource, property, otherResource)):
        print(f"{s} {p} {o}")

In [9]:
schema = rdf.Graph()
schema.parse("Lab3_RDFS_graph.ttl")

kevin = rdf.URIRef("http://example.com/kevin")
angela = rdf.URIRef("http://example.com/angela")

print("Triples where Kevin is the subject:");
print_graph(schema, kevin, None, None)

print("\nTriples where Angela is the subject:");
print_graph(schema, angela, None, None)

Triples where Kevin is the subject:
http://example.com/kevin http://example.com/hasMother http://example.com/angela
http://example.com/kevin http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://example.com/Teenager
http://example.com/kevin http://example.com/hasAge 17

Triples where Angela is the subject:
http://example.com/angela http://example.com/isMotherOf http://example.com/kevin


In [10]:
owlrl.DeductiveClosure(owlrl.RDFS_Semantics).expand(schema)

print("Triples where Kevin is the subject:");
print_graph(schema, kevin, None, None)

print("\nTriples where Angela is the subject:");
print_graph(schema, angela, None, None)


Triples where Kevin is the subject:
http://example.com/kevin http://example.com/hasMother http://example.com/angela
http://example.com/kevin http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://example.com/Teenager
http://example.com/kevin http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Resource
http://example.com/kevin http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://example.com/Adolescent
http://example.com/kevin http://example.com/hasAge 17
http://example.com/kevin http://example.com/hasParent http://example.com/angela

Triples where Angela is the subject:
http://example.com/angela http://example.com/isMotherOf http://example.com/kevin
http://example.com/angela http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Resource
http://example.com/angela http://example.com/loves http://example.com/kevin


**Make a note of the triples printed above.**

#### B. Tasks
You will notice that each triple in the "Schema" section of ``Lab3_RDFS_graph.ttl`` has a lab task number associated with it. For each task in this lab, you must identify which triple(s) in the "Schema" section are associated with that task and then uncomment it to see its effect on the RDFS inferences obtained.

To uncomment a triple, remove only the **first** ``#`` character on the line for that triple. **Do not remove the second ``#`` character on the line**. After each modification, save ``Lab3_RDFS_graph.ttl`` and then re-run the code in **Cell 3A. Build RDFS model and print triples**

1. Uncomment the triples associated with ``Lab 3 - Task 1`` and observe the effect on the inferences. Do you see implicit triples (triples that are not written down in ``Lab3_RDFS_graph.ttl``) printed in the output? If so, what are these inferred triples? Why do you think these triples are inferred?
2. Uncomment the triples associated with ``Lab 3 - Task 2`` and observe the effect on the inferences. Do you see implicit triples (triples that are not written down in ``Lab3_RDFS_graph.ttl``) printed in the output? If so, what are these inferred triples? Why do you think these triples are inferred?
3. Uncomment the triples associated with ``Lab 3 - Task 3`` and observe the effect on the inferences. Do you see implicit triples (triples that are not written down in ``Lab3_RDFS_graph.ttl``) printed in the output? If so, what are these inferred triples? Why do you think these triples are inferred?

#### C. Interested in what your graph looks like visually?

Copy and paste your Turtle code into the textbox located on this page: [http://www.ldf.fi/service/rdf-grapher](http://www.ldf.fi/service/rdf-grapher) and click on the ``Visualize`` button. You can use this service just out of interest. If it does not work, that's okay - it is not required that it does. We are not going to provide support on how to make it work - so do not ask questions about this. Focus on the lab tasks in **3B.**, they are much more important. [RDF-grapher](http://www.ldf.fi/service/rdf-grapher) does not produce the prettiest visualisations but is generally reliable for producing functional visualisations for smaller RDF graphs.

**Side note:** some of you may have noticed that the inferences we are making here are [deductive](https://en.wikipedia.org/wiki/Deductive_reasoning) and may be wondering if the Semantic Web vision encourages and applies [inductive reasoning](https://en.wikipedia.org/wiki/Inductive_reasoning) methods (such as Machine and Deep Learning) as well. The answer is an emphatic **Yes!**. We teach students how to apply these inductive techniques in our Masters course at DKE on [Building and Mining Knowledge Graphs](https://www.maastrichtuniversity.nl/meta/371276/building-and-mining-knowledge-graphs) which builds on many of the technologies we cover in this course such as RDF, [SPARQL](https://www.w3.org/TR/rdf-sparql-query/) and [OWL](https://www.w3.org/TR/owl2-overview/).